# Perform vertical, horizonal, semantic data integration on healthcare dataset

In [1]:
import pandas as pd

df_vitals = pd.read_csv('patient_vitals.csv')
df_labs = pd.read_csv('lab_results.csv')

In [2]:
# Horizontal Merging
df_horizontal_merge = pd.merge(
    df_vitals,
    df_labs,
    on='Patient_ID',
    how='inner'
)

In [3]:
df_horizontal_merge.head(5)

,Patient_ID,Age_Years,Gender,Weight_kg,Height_cm,Diagnosis_Code,Hospital_Location,Test_Date,Test_Name,Result_Value,Result_Unit,Medication_Name,Dosage_mg
0,1000,63,Male,76.9,162.6,Asthma,North Campus,2024-12-14,Cholesterol,161.1,mmol/L,Metform.,279
1,1001,76,Other,99.9,163.9,Asthma,North Campus,2024-01-15,Cholesterol,106.3,mmol/L,Lisinopril,70
2,1002,53,Male,98.9,164.7,HTN,South Campus,2024-10-19,Hemoglobin,NaN,mg/dL,Lisinopril,152
3,1003,39,Female,63.3,180.5,Asthma,South Campus,2024-08-03,Glucose,130.3,mg/dL,Metformin,197
4,1004,67,Female,83.0,159.1,HTN,North Campus,2024-02-16,Cholesterol,236.6,mg/dL,Lisi.,432


In [4]:
# Vertical Merging
df_new_patients = df_labs[['Patient_ID']].drop_duplicates()
df_new_patients['Source'] = 'Lab Results'

In [5]:
df_existing_patients = df_vitals[['Patient_ID', 'Hospital_Location']].drop_duplicates()
df_existing_patients['Source'] = 'Vitals'

In [6]:
df_vertical_merge = pd.concat([df_existing_patients, df_new_patients], ignore_index=True)
df_vertical_merge.drop_duplicates(subset=['Patient_ID'], keep='first', inplace=True)

In [7]:
df_vertical_merge.head(5)

,Patient_ID,Hospital_Location,Source
0,1000,North Campus,Vitals
1,1001,North Campus,Vitals
2,1002,South Campus,Vitals
3,1003,South Campus,Vitals
4,1004,North Campus,Vitals


In [8]:
# Semantic Merging
diagnosis_mapping = {
    'HTN': '44054006 (Hypertension)',
    'DM-II': '73211009 (Diabetes Mellitus Type 2)',
    'Asthma': '195967001 (Asthma)',
    'Migr.': '59368008 (Migraine)'
}

In [9]:
df_vitals['Diagnosis_Code_Standard'] = df_vitals['Diagnosis_Code'].replace(diagnosis_mapping)

In [11]:
import numpy as np

medication_mapping = {
    'Lisi.': 'Lisinopril',
    'Metform.': 'Metformin',
    'Simbast.': 'Simvastatin',
    'N/A': np.nan
}

In [12]:
df_labs['Medication_Name_Standard'] = df_labs['Medication_Name'].replace(medication_mapping)

In [13]:
gender_mapping = {
    'F': 'Female',
    'M': 'Male'
}
df_vitals['Gender_Standard'] = df_vitals['Gender'].replace(gender_mapping)

In [14]:
df_vitals.head(5)

,Patient_ID,Age_Years,Gender,Weight_kg,Height_cm,Diagnosis_Code,Hospital_Location,Diagnosis_Code_Standard,Gender_Standard
0,1000,63,Male,76.9,162.6,Asthma,North Campus,195967001 (Asthma),Male
1,1001,76,Other,99.9,163.9,Asthma,North Campus,195967001 (Asthma),Other
2,1002,53,Male,98.9,164.7,HTN,South Campus,44054006 (Hypertension),Male
3,1003,39,Female,63.3,180.5,Asthma,South Campus,195967001 (Asthma),Female
4,1004,67,Female,83.0,159.1,HTN,North Campus,44054006 (Hypertension),Female
